## 简答题

1. TensorFlow 是否可以简单替代 NumPy？两者之间的主要区别是什么？

不能，numpy是立即执行，TensorFlow是通过图

2. 使用 `tf.range(10)` 和 `tf.constant(np.arange(10))` 是否会得到相同的结果？

会

3. 可以通过编写函数或继承 `tf.keras.losses.Loss` 来定义自定义损失函数。两种方法分别应该在什么时候使用？

函数适合不需要保存或配置参数的一次性使用的损失函数
继承适合需要配置参数，要保存状态的可复用的复杂损失函数

4. 可以直接在函数中定义自定义指标或采用 `tf.keras.metrics.Metric` 子类。两种方法分别应该在什么时候使用？

函数适合简单一次性使用不累积的指标
继承适合需要跨批次累积，需要重制状态的指标

5. 什么时候应该自定义层而不是自定义模型？

需要在不同模型中重复使用特定功能，需要自定义的前向传播和训练过程

6. 有哪些示例需要编写自定义训练循环？

多优化器训练，特殊的梯度操作

7. 自定义 Keras 组件中可以包含任意 Python 代码，还是必须转换为 TF 函数？

在图模式下，函数可以转换为 TF 函数，但必须使用 `@tf.function` 装饰器。

8. 如果要将函数转换为 TF 函数，应避免哪些主要模式？

避免使用python原生控制流，使用tensor类型

9. 何时需要创建动态 Keras 模型？ 如何动态创建Keras模型？为什么不是所有模型都动态化？

输入形状在运行时才能确定，大多数场景不需要


## 编程题

1. 实现一个执行层归一化的自定义层：
    - a. `build()` 方法应定义两个可训练的权重 α 和 β，它们的形状均为 `input_shape[-1:]`，数据类型为 `tf.float32`。α 应该用 1 初始化，而 β 必须用 0 初始化。
    - b. `call()` 方法应计算每个实例特征的均值和标准差。为此，可以使用 `tf.nn.moments(inputs, axes=-1, keepdims=True)`，它返回同一实例的均值 μ 和方差 σ²（计算方差的平方根便可获得标准差）。然后，该函数应计算并返回
      $$
      \alpha \otimes \frac{(X-\mu)}{(\sigma+\epsilon)} + \beta
      $$
      其中 ε 是表示项精度的一个常量（避免被零除的小常数，例如 0.001）,$\otimes$表示逐个元素相乘
    - c. 确保自定义层产生与tf.keras.layers.LayerNormalization层相同（或几乎相同）的输出。

2. 使用自定义训练循环训练模型来处理Fashion MNIST数据集（13_神经网络介绍 里用的数据集）：

    - a.显示每个轮次、迭代、平均训练损失和每个轮次的平均精度（在每次迭代中更新），以及每个轮次结束时的验证损失和精度。
    - b.尝试对上面的层和下面的层使用具有不同学习率的不同优化器。

In [3]:
import tensorflow as tf
import numpy as np
class myLayer(tf.keras.layers.Layer):
    def __init__(self, units, activation=None, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = tf.keras.activations.get(activation)
        
    def build(self, input_shape):
        self.alpha = self.add_weight(name='alpha', shape=input_shape[-1:], dtype=tf.float32, initializer='ones', trainable=True)
        self.beta = self.add_weight(name='beta', shape=input_shape[-1:], dtype=tf.float32, initializer='zeros', trainable=True)
        super().build(input_shape)
        
    def call(self, X):
        mean, std2 = tf.nn.moments(X, axes=-1, keepdims=True)
        std = tf.sqrt(std2)
        n = (X - mean)/(std + 1e-3)
        result = self.alpha * n + self.beta
        return result
    
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "units": self.units, "activation": tf.keras.activations.serialize(self.activation)}

In [8]:
from sklearn.model_selection import train_test_split
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


In [9]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    myLayer(100, activation='relu'),
    myLayer(100, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

C:\Users\31752\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [30]:

def random_batch(X,y, batch_size=32):
    idx = np.random.randint(len(X), size=batch_size)
    return X[idx], y[idx]


def print_status_bar(step, total, loss, accuracy, metrics=None):
    # 构建指标字符串，包含loss和accuracy
    metrics_parts = [f"{loss.name}: {loss.result():.4f}", f"accuracy: {accuracy.result():.4f}"]
    
    # 添加其他指标（如果有的话）
    if metrics:
        for metric in metrics:
            metrics_parts.append(f"{metric.name}: {metric.result():.4f}")
    
    metrics_str = " - ".join(metrics_parts)
    end = "" if step < total else "\n"
    print(f"\r{step}/{total} - " + metrics_str, end=end)
  

In [31]:
upper_layer = model.get_layer("my_layer_1")
lower_layer = model.get_layer("my_layer")
upper_layer.trainable_variables

[<Variable path=sequential/my_layer_1/alpha, shape=(784,), dtype=float32, value=[1.0051452  1.0057523  1.0175804  1.0038154  0.99863946 0.9969828
  0.99815476 1.009924   1.0194819  1.03052    1.0561574  1.0212556
  1.0249522  1.0181849  1.0531468  1.0082369  1.0011609  1.0324738
  1.019143   1.0699021  1.0170674  1.0216497  1.0047284  1.0124546
  1.0036169  1.0046781  1.0156035  0.99404997 1.0086076  0.9904847
  1.0026972  0.99379635 0.9973847  1.0161239  1.0321069  1.0009173
  1.0055964  1.028414   1.0212828  1.0004395  1.0530629  0.99210227
  0.9835926  0.98956543 1.0050054  1.0180683  1.0104201  1.0156282
  1.0099325  1.0148365  1.0057255  1.0153649  1.0011528  0.98843366
  0.9964974  0.9996796  1.0076636  0.98958397 1.0143368  1.002252
  1.013004   0.9989664  1.0061308  1.010861   1.0266836  0.9979819
  0.9908797  1.0209215  1.0133418  0.99241567 1.0173812  1.0192477
  1.0225645  1.0057298  1.0118144  0.99173665 1.0172216  1.0162024
  1.0199205  1.0239184  0.9935892  0.99184215 0.9

In [32]:

n_epochs = 5
batch_size = 32
n_steps = len(X_train) // batch_size

optimizer1 = tf.keras.optimizers.SGD(learning_rate=0.01)
optimizer2 = tf.keras.optimizers.SGD(learning_rate=0.05)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
train_loss = tf.keras.metrics.Mean(name="mean_loss")
val_loss = tf.keras.metrics.Mean(name="val_loss")
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name="train_accuracy")
val_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name="val_accuracy")
metrics = [tf.keras.metrics.SparseCategoricalCrossentropy()]


for epoch in range(1, n_epochs + 1):
    print(f"Epoch {epoch}/{n_epochs}")
    for step in range(1, n_steps + 1):
        X_batch, y_batch = random_batch(X_train, y_train)

        with tf.GradientTape(persistent=True) as tape:
            y_pred = model(X_batch, training=True)
            main_loss = tf.reduce_mean(loss_fn(y_batch, y_pred))  # 
            loss = tf.add_n([main_loss] + model.losses)  
        upper_layer = model.get_layer("my_layer_1")
        lower_layer = model.get_layer("my_layer")
        gradients1 = tape.gradient(loss, upper_layer.trainable_variables)
        gradients2 = tape.gradient(loss, lower_layer.trainable_variables)
        del tape
        
        optimizer1.apply_gradients(zip(gradients1, upper_layer.trainable_variables))
        optimizer2.apply_gradients(zip(gradients2, lower_layer.trainable_variables))
        for variable in model.variables:
            if variable.constraint is not None:
                variable.assign(variable.constraint(variable))

        train_loss(loss)  
        train_accuracy(y_batch, y_pred)
        for metric in metrics:
            metric(y_batch, y_pred)
        print_status_bar(step, n_steps, train_loss, train_accuracy, metrics)
    
    val_loss(model(X_val, training=False))
    val_accuracy(y_val, model(X_val, training=False))
    print(f"Validation: loss: {val_loss.result():.4f}, accuracy: {val_accuracy.result():.4f}")
    for metric in [train_loss] + metrics:
        metric.reset_state()  

Epoch 1/5
1500/1500 - mean_loss: 0.3948 - accuracy: 0.8639 - sparse_categorical_crossentropy: 0.3948
Validation: loss: 0.1000, accuracy: 0.8480
Epoch 2/5
1500/1500 - mean_loss: 0.3997 - accuracy: 0.8633 - sparse_categorical_crossentropy: 0.3997
Validation: loss: 0.1000, accuracy: 0.8481
Epoch 3/5
1500/1500 - mean_loss: 0.3927 - accuracy: 0.8640 - sparse_categorical_crossentropy: 0.3927
Validation: loss: 0.1000, accuracy: 0.8483
Epoch 4/5
1500/1500 - mean_loss: 0.3943 - accuracy: 0.8639 - sparse_categorical_crossentropy: 0.3943
Validation: loss: 0.1000, accuracy: 0.8483
Epoch 5/5
1500/1500 - mean_loss: 0.3919 - accuracy: 0.8637 - sparse_categorical_crossentropy: 0.3919
Validation: loss: 0.1000, accuracy: 0.8485
